# Initial

In [1]:
from sklearn.metrics.cluster import v_measure_score, silhouette_score
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import time
from sklearn.cluster import MiniBatchKMeans
import warnings
warnings.filterwarnings("ignore")

In [2]:
'''
EXTERNAL KERNEL
'''
google_colab = False
kaggle = False

'''
CUDA
'''
cuda = False

'''
DATA REPRESENTATION

1 => SINGLE READ | 2 => ADD FEATURES | 3 => WINDOW TO FEATURES
'''
DATA_REPRESENTATION = 2

'''
DOWNSAMPLE FACTOR

1 => 10hz *original rate* | 2 => 5Hz | 5 => 2Hz | 10 => 1hz
'''
DOWNSAMPLE_FACTOR = 5

'''
WINDOWS LENGHT

* needs divisor by datapoints target
* considering downsample factor = 5

1 => WINDOW DISABLED | 2 => 1 second | 4 => 2 seconds | 10 => 5 seconds | 20 => 10 seconds | 200 => 100 seconds *full flight*
'''
WINDOW_LENGHT =  1


'''
LIMITADOR

Quantity of samples in the execution of the tests.
'''
LIMITADOR = 500

'''
LOSS FACTOR [0,1]

Ignores outliers in calculating the stats of losses in regenerated data.
'''
LOSS_FACTOR = 1

'''
TRAIN_SIZE [0,1]

Percentage of samples to be trained
'''
TRAIN_SIZE = 0.8

'''
OUTPUT_FILE_NAME

File with output results
'''
OUTPUT_FILE_NAME = 'output_kmeans_ds_dr_' + str(DATA_REPRESENTATION) + '-ts_' + str(TRAIN_SIZE) + '-lf_' + str.replace(str(LOSS_FACTOR), '.', '') + '-limit_' + str(LIMITADOR) + '-wl_' + str(WINDOW_LENGHT) + '.txt'

'''
PATH_OUTPUTS

local : ./outputs/
google colab : /content/drive/My Drive/
'''
if google_colab:
    PATH_OUTPUTS = '/content/drive/My Drive/'
elif kaggle:    
    PATH_OUTPUTS = ''
else:
    PATH_OUTPUTS = './outputs/'


'''
PATH_DATASET

'''
PATH_DATASET = '../dataset/original/'

'''
FLUSH FILE

If output results file is ON
'''
FLUSH_FILE = False

In [3]:
if google_colab:
    !pip install git+https://github.com/online-ml/river --upgrade

    from google.colab import drive

    drive.mount('/content/drive')
    path = '/content/drive/My Drive/ACADÊMICO/MESTRADO/DISSERTAÇÃO/CHAPTERS/5 EXPERIMENTO/dataset/data_representation_1'
    dict_ds_original = {
        'data_ds3_normal_t1_original' : pd.read_csv(path+'/F16_DS3_normal_t1.csv', header=None),
        'data_ds3_normal_t2_original' : pd.read_csv(path+'/F16_DS3_normal_t2.csv', header=None),
        'data_ds3_fault1_original' : pd.read_csv(path+'/F16_DS3_fault1_leakage.csv', header=None),
        'data_ds3_fault2_original' : pd.read_csv(path+'/F16_DS3_fault2_viscousfriction.csv', header=None),
        'data_ds3_fault3_original' : pd.read_csv(path+'/F16_DS3_fault3_compressibility.csv', header=None),
        'data_ds3_fault4_original' : pd.read_csv(path+'/F16_DS3_fault4_fixedposition.csv', header=None),
    }
elif kaggle:
    !conda install -y gdown
    !gdown https://drive.google.com/u/0/uc?id=1G88okIVmdcgLFlmd7rDRhHvHv98yK3UB
    !gdown https://drive.google.com/u/0/uc?id=1fX3utfHMjwKTt7IW4D01bnm-hv88yzrJ 
    !gdown https://drive.google.com/u/0/uc?id=1yUG3R5zK2AIxtS9Q4Fk-udkKBZeYShgb
    !gdown https://drive.google.com/u/0/uc?id=1OBRDtuqNEZ-3Z-q0helWh2xGiAxeLACH
    !gdown https://drive.google.com/u/0/uc?id=17oDi60sWYsWHHxzj2aA9m6ARm8zQ81m_
    !gdown https://drive.google.com/u/0/uc?id=1jKEK4s5sYJh8PHtpHeV8ABOsHjuB26RA

    dict_ds_original = {
        'data_ds3_normal_t1_original' : pd.read_csv('F16_DS3_normal_t1.csv', header=None),
        'data_ds3_normal_t2_original' : pd.read_csv('F16_DS3_normal_t2.csv', header=None),
        'data_ds3_fault1_original' : pd.read_csv('F16_DS3_fault1_leakage.csv', header=None),
        'data_ds3_fault2_original' : pd.read_csv('F16_DS3_fault2_viscousfriction.csv', header=None),
        'data_ds3_fault3_original' : pd.read_csv('F16_DS3_fault3_compressibility.csv', header=None),
        'data_ds3_fault4_original' : pd.read_csv('F16_DS3_fault4_fixedposition.csv', header=None),
    }
else:
    dict_ds_original = {
        'data_ds3_normal_t1_original' : pd.read_csv(PATH_DATASET+'F16_DS3_normal_t1.csv', header=None),
        'data_ds3_normal_t2_original' : pd.read_csv(PATH_DATASET+'F16_DS3_normal_t2.csv', header=None),
        'data_ds3_fault1_original' : pd.read_csv(PATH_DATASET+'F16_DS3_fault1_leakage.csv', header=None),
        'data_ds3_fault2_original' : pd.read_csv(PATH_DATASET+'F16_DS3_fault2_viscousfriction.csv', header=None),
        'data_ds3_fault3_original' : pd.read_csv(PATH_DATASET+'F16_DS3_fault3_compressibility.csv', header=None),
        'data_ds3_fault4_original' : pd.read_csv(PATH_DATASET+'F16_DS3_fault4_fixedposition.csv', header=None),
    }

# Dataset

In [4]:
dict_ds = dict_ds_original.copy()

if dict_ds['data_ds3_normal_t1_original'].shape[0] % DOWNSAMPLE_FACTOR != 0 or dict_ds['data_ds3_fault1_original'].shape[0] % DOWNSAMPLE_FACTOR != 0:
    raise Exception('Needs to be ?shape? divisor')

for n, dataset_name in enumerate(dict_ds):
    dataset = dict_ds[dataset_name].to_numpy()

    downsampled = dataset[::DOWNSAMPLE_FACTOR]

    x, y = downsampled.shape

    # resample
    dict_ds[dataset_name] = pd.DataFrame(downsampled.reshape((int(x/WINDOW_LENGHT),y*WINDOW_LENGHT)))


In [5]:
# ADD COLUMNS WITH DIFF PREVIOUS VALUES

if (DATA_REPRESENTATION == 2):
    frame_size = int(1000/DOWNSAMPLE_FACTOR)

    for n, dataset_name in enumerate(dict_ds):
        dataset = dict_ds[dataset_name].to_numpy()

        dimension = dataset.shape[1]
        samples = dataset.shape[0]

        # GENERATE NEW DIMENSIONS
        dataset = np.concatenate((dataset, np.zeros((samples,dimension))), axis=1)

        for f in np.arange(0,int(samples/frame_size)):
            # OBTAIN THE FRAME FLIGHT
            frame = dataset[f*frame_size:(f+1)*frame_size, 0:dimension]

            # CALCULATE DIFFERENCE
            chunk = np.diff(frame, axis=0)

            # DONT CALCULATE THE DIFFERENCE FOR EACH FIRST TIMESTEP
            chunk = np.insert(chunk, 0, frame[0, 0:dimension], axis=0)

            # UPDATE DATASET WITH NEW FRAME INTO NEW DIMENSIONS
            dataset[f*frame_size:(f+1)*frame_size,dimension:dimension*2] = chunk

        dict_ds[dataset_name] = pd.DataFrame(dataset)

# Run

In [6]:
log = None
if FLUSH_FILE:
    log = open(PATH_OUTPUTS+OUTPUT_FILE_NAME, "a", buffering=1)

options = {'n1': [dict_ds['data_ds3_normal_t1_original'], 0, [0]],
           'n2': [dict_ds['data_ds3_normal_t2_original'], 0, [0]],
           'f1': [dict_ds['data_ds3_fault1_original'], 1, [1]],
           'f2': [dict_ds['data_ds3_fault2_original'], 1, [2]],
           'f3': [dict_ds['data_ds3_fault3_original'], 1, [3]],
           'f4': [dict_ds['data_ds3_fault4_original'], 1, [4]]
}

samples = 2000
nc = 100
bs = 16
n_folds = 10
scenarios = [
    ['n1', 'n2', 'f2', 'f3'], # scenario 1
    ['f1', 'f2', 'f3', 'f4'], # scenario 2
    ['n1', 'f1', 'f2', 'f3'], # scenario 3
    ['n2', 'f2', 'f4'], # scenario 4
    ['n1', 'f1', 'f2', 'f4'], # scenario 5
 ]
outputs = np.zeros((n_folds, 2))

s = 0
for scenario in scenarios:
    s = s + 1
    print ('kmeans', s, scenario, nc, bs, file=log)

    for fold in np.arange(0,n_folds):

        kmeans = MiniBatchKMeans(n_clusters=nc, batch_size=bs, random_state=42)
        ss = StandardScaler()
        y_pred = []

        # get random indexes for each dataset, but the datasets are ordered
        idxs = np.random.randint(0,100000,samples)

        data_x = options[scenario[0]][0].iloc[idxs, :]
        data_y = options[scenario[0]][2]*samples
        for x in np.arange(1,len(scenario)):
            data_x = np.concatenate((data_x, options[scenario[x]][0].iloc[idxs, :]))
            data_y = np.concatenate((data_y, options[scenario[x]][2]*samples))

        data = np.concatenate((data_x, data_y.reshape(-1,1)), axis=1)

        batch = np.split(data, int(len(data_x)/nc))

        for x in batch:
            x_nd = np.array(x)[:,:-1]

            # standardization
            ss.partial_fit(x_nd)
            x_trans = ss.transform(x_nd)

            # train
            kmeans.partial_fit(x_trans)

            # test
            for pred in kmeans.predict(x_trans):
              y_pred.append(pred)

        outputs[fold][0] = silhouette_score(ss.transform(data[:,:-1]), y_pred)
        outputs[fold][1] = v_measure_score(data_y, y_pred, beta=0) # 0 plus homogeneity | 2 completeness

    print(outputs)

if FLUSH_FILE:
    log.close()

kmeans 1 ['n1', 'n2', 'f2', 'f3'] 100 16
[[-5.07458337e-02  8.68317523e-01]
 [ 4.01824935e-02  8.47852277e-01]
 [-4.95477311e-02  8.81304894e-01]
 [ 7.76332757e-04  8.77198151e-01]
 [-4.07186642e-02  8.34441203e-01]
 [ 1.96198007e-02  8.75483877e-01]
 [-7.52617283e-02  8.68133786e-01]
 [ 7.14909673e-03  8.56060454e-01]
 [-2.03453452e-02  8.81461991e-01]
 [-3.44650680e-02  9.01211831e-01]]
kmeans 2 ['f1', 'f2', 'f3', 'f4'] 100 16
[[-0.04741696  0.86076406]
 [ 0.01719381  0.85478537]
 [ 0.02629305  0.83193706]
 [-0.00828034  0.8688723 ]
 [-0.01669726  0.82449933]
 [-0.00878547  0.86202044]
 [-0.03455398  0.85844692]
 [-0.02726312  0.84081929]
 [-0.01464865  0.84985155]
 [-0.04551974  0.82199585]]
kmeans 3 ['n1', 'f1', 'f2', 'f3'] 100 16
[[ 0.00715837  0.83413108]
 [-0.00988626  0.7983028 ]
 [-0.01253003  0.86910811]
 [ 0.09184286  0.85287718]
 [-0.06538223  0.83851098]
 [-0.00864709  0.80670133]
 [ 0.0126575   0.82207482]
 [-0.03485228  0.81230508]
 [-0.08453269  0.84879107]
 [ 0.0018677

In [7]:
# generate 3 scenarios
from itertools import permutations

sets = []

caracteres = ['n1', 'n2', 'f1', 'f2', 'f3', 'f4']
for l in np.arange(2,6):
    for subset in permutations(caracteres, int(l)):
        sets.append(subset)

sets.append(tuple(caracteres))

sets = np.array(sets)
choose = sets[[np.random.randint(0, len(sets), 10)]]